In [1]:
import sys

# setting path
sys.path.append('..')


In [2]:
# outside package
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Read Data

In [18]:
heroes_data = pd.read_csv("../data/processed_data/heroes_data.csv", index_col=0)
heroes_data.head()

,id,name,win_rate,against_hero_1,against_hero_2,against_hero_3,against_hero_4,against_hero_5,against_hero_6,against_hero_7,...,against_hero_119,against_hero_120,against_hero_121,against_hero_123,against_hero_126,against_hero_128,against_hero_129,against_hero_135,against_hero_136,against_hero_137
0,1,Anti-Mage,0.489100,NaN,0.566667,0.464789,0.447368,0.441860,0.608696,0.588235,...,0.545455,0.500000,0.535433,0.381579,0.477273,0.460526,0.413793,0.428571,0.520000,0.521739
1,2,Axe,0.485514,0.433333,NaN,0.474576,0.576923,0.471264,0.507937,0.666667,...,0.473684,0.482143,0.474227,0.508475,0.486486,0.490196,0.544828,0.419355,0.471429,0.392857
2,3,Bane,0.552398,0.535211,0.525424,NaN,0.605042,0.576087,0.495413,0.562500,...,0.516129,0.530928,0.592018,0.617925,0.616822,0.595745,0.583984,0.547945,0.490066,0.589744
3,4,Bloodseeker,0.496582,0.552632,0.423077,0.394958,NaN,0.589147,0.480769,0.490909,...,0.480000,0.515513,0.620321,0.537313,0.567164,0.506550,0.517647,0.517413,0.483871,0.387755
4,5,Crystal Maiden,0.496833,0.558140,0.528736,0.423913,0.410853,NaN,0.491124,0.461538,...,0.519481,0.507519,0.550296,0.516807,0.400000,0.482085,0.538462,0.492593,0.490141,0.514851


In [19]:
heroes_stats = pd.read_csv("../data/processed_data/heroes_stats.csv", index_col=0)
heroes_stats.head()

,id,localized_name,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_mr,base_attack_min,base_attack_max,...,Durable,Escape,Initiator,Jungler,Nuker,Pusher,Support,agi,int,Melee
0,1,Anti-Mage,200,0.25,75,0.0,0.0,25,29,33,...,0,1,0,0,1,0,0,1,0,1
1,2,Axe,200,2.50,75,0.0,-1.0,25,27,31,...,1,0,1,1,0,0,0,0,0,1
2,3,Bane,200,0.25,75,0.0,1.0,25,33,39,...,1,0,0,0,1,0,1,0,1,0
3,4,Bloodseeker,200,0.25,75,0.0,2.0,25,35,41,...,0,0,1,1,1,0,0,1,0,1
4,5,Crystal Maiden,200,0.25,75,1.0,-1.0,25,28,34,...,0,0,0,1,1,0,1,0,1,0


In [20]:
data = pd.read_csv("../data/processed_data/data.json", index_col=0)
data.head()

,match_id,radiant_win,region,patch,first_pick_team,selection_0,selection_1,selection_2,selection_3,selection_4,...,selection_14,selection_15,selection_16,selection_17,selection_18,selection_19,selection_20,selection_21,selection_22,selection_23
0,5746455668,False,3,47,1,85,91,79,41,43,...,47,20,64,94,128,65,4,33,28,55
1,5795993534,True,3,47,0,65,88,32,111,13,...,38,51,12,4,47,54,46,11,55,114
2,5883814830,True,5,47,1,102,88,73,13,86,...,129,121,110,32,113,106,126,17,39,74
3,5860105982,True,17,47,0,38,88,9,13,111,...,102,42,126,11,70,29,12,60,104,18
4,5831784791,True,3,47,1,7,90,88,91,111,...,86,107,94,1,28,49,2,22,55,104


In [21]:
merged_data = pd.read_csv("../data/processed_data/merged_data.csv", index_col=0)
merged_data.head()

,match_id,radiant_win,region,patch,first_pick_team,selection_0_id,selection_0_base_health,selection_0_base_health_regen,selection_0_base_mana,selection_0_base_mana_regen,...,selection_23_Durable,selection_23_Escape,selection_23_Initiator,selection_23_Jungler,selection_23_Nuker,selection_23_Pusher,selection_23_Support,selection_23_agi,selection_23_int,selection_23_Melee
0,5746455668,False,3,47,1,85,200,0.25,75,0.25,...,0,1,1,1,0,0,0,0,1,1
1,6111186894,False,3,48,0,110,200,2.00,75,0.00,...,0,1,1,1,0,0,0,0,1,1
2,6054787486,False,5,48,0,46,200,0.25,75,0.00,...,0,1,1,1,0,0,0,0,1,1
3,5848322719,True,3,47,0,88,200,2.00,75,0.00,...,0,1,1,1,0,0,0,0,1,1
4,5807290727,True,3,47,0,129,200,0.25,75,0.50,...,0,1,1,1,0,0,0,0,1,1


# Prepare Data For Training

In [22]:
X = merged_data.drop(["match_id", "radiant_win"], axis=1)
y = merged_data["radiant_win"]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

# Neural Network

In [24]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Convert Data to Tensor

In [25]:
training_data = TensorDataset(torch.Tensor(X_train.values), torch.Tensor(y_train.values))
test_data = TensorDataset(torch.Tensor(X_test.values), torch.Tensor(y_test.values))

Create Data Loader

In [26]:
training_dataloader = DataLoader(training_data, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=10, shuffle=True)

In [27]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C]: torch.Size([10, 819])
Shape of y: torch.Size([10]) torch.float32


In [28]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(819, 2000),
            nn.ReLU(),
            nn.Linear(2000, 512),
            nn.ReLU(),
            nn.Linear(512, 200),
            nn.ReLU(),
            nn.Linear(200, 100)
        )


    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=819, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=200, bias=True)
    (5): ReLU()
    (6): Linear(in_features=200, out_features=100, bias=True)
  )
)


In [29]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [30]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = y.type(torch.LongTensor)   # casting to long
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [31]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    # optimize calculation time
    with torch.no_grad():
        for X, y in dataloader:
            y = y.type(torch.LongTensor)   # casting to long
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [32]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 21.954571  [    0/ 7289]
loss: 0.775445  [ 1000/ 7289]
loss: 0.590973  [ 2000/ 7289]
loss: 0.680393  [ 3000/ 7289]
loss: 0.741875  [ 4000/ 7289]
loss: 0.678084  [ 5000/ 7289]
loss: 0.628326  [ 6000/ 7289]
loss: 0.692145  [ 7000/ 7289]
Test Error: 
 Accuracy: 55.0%, Avg loss: 0.691102 

Epoch 2
-------------------------------
loss: 0.697704  [    0/ 7289]
loss: 0.710078  [ 1000/ 7289]
loss: 0.679697  [ 2000/ 7289]
loss: 0.769248  [ 3000/ 7289]
loss: 0.712573  [ 4000/ 7289]
loss: 0.632314  [ 5000/ 7289]
loss: 0.706924  [ 6000/ 7289]
loss: 0.701645  [ 7000/ 7289]
Test Error: 
 Accuracy: 55.0%, Avg loss: 0.690448 

Epoch 3
-------------------------------
loss: 0.684545  [    0/ 7289]
loss: 0.679297  [ 1000/ 7289]
loss: 0.683830  [ 2000/ 7289]
loss: 0.696316  [ 3000/ 7289]
loss: 0.653536  [ 4000/ 7289]
loss: 0.648249  [ 5000/ 7289]
loss: 0.702135  [ 6000/ 7289]
loss: 0.679990  [ 7000/ 7289]
Test Error: 
 Accuracy: 55.0%, Avg loss: 0.688292 

Epo

## Save Model

In [33]:
torch.save(model.state_dict(), "../models/draft_evaluation_model.pth")

In [34]:
model.load_state_dict(torch.load('../models/draft_evaluation_model.pth'))

<All keys matched successfully>

In [35]:
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=819, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=200, bias=True)
    (5): ReLU()
    (6): Linear(in_features=200, out_features=100, bias=True)
  )
)

# Random Forest


In [43]:
data["radiant_win"].mean()


0.5266681299385426